<a href="https://colab.research.google.com/github/Kavya2099/recommendation-system/blob/main/Project_2_Content_based_Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing data


1.   Get file
2.   Unzipping the zip file
3.   Reading file



In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip





--2022-10-25 02:51:56--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  77.0MB/s    in 3.2s    

2022-10-25 02:52:02 (77.0 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [ ]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
!head ml-25m/movies.csv

movie= pd.read_csv('ml-25m/movies.csv')
ratings = pd.read_csv('ml-25m/ratings.csv')

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


In [ ]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


We will recommend the movies based on the title provided. It is a content based filtering model.

Filtering is based on the title. 

As we can see we have other characters involved in movie, we'll change the movie title name to be easily used by ML


A regular expression (or RE) specifies a set of strings that matches it; the functions let you check if a particular string matches a given regular expression (or if a given regular expression matches a particular string, which comes down to the same thing)

In [ ]:
import re

def new_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title
movie["new_title"] = movie["title"].apply(new_title)

In [ ]:
movie.shape

(62423, 4)

In [ ]:
movie

,movieId,title,genres,new_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


Using TfidfVectorizer


Transforms text to feature vectors that can be used as input to estimator

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

""" An n-gram is just a string of n words in a row. E.g. the sentence 'I am Groot' contains the 2-grams 'I am' and 'am Groot'.
 The sentence is itself a 3-gram. Set the parameter ngram_range=(a,b) where a is the minimum and b is the maximum size of ngrams you want to include in your features """


vectorizer = TfidfVectorizer(stop_words="english",ngram_range=(1,2))

#in our case we are using min 1 and max 2 size of grams

tfidf = vectorizer.fit_transform(movie["new_title"])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendation(title):
  
  #passing to new_title function to clean up
  title = new_title(title)
  #transfering the title; since it is a string and we have to pass a list we are using [title]
  query_vec = vectorizer.transform([title])
  #print([title])
  #print(query_vec)
  #passing inside cosine similarity to get the similarity value. We'll have only one value as we have compared with one string title
  similarity = cosine_similarity(query_vec, tfidf).flatten()
  #print(similarity)
  #sim_scores= enumerate(similarity)
  #print(sim_scores)


  #sorting the similairty values and getting the index of those values using argsort; we can use argpartition also
  inx = np.argsort(similarity)

  #getting the last 10 values after sorting and saving in descending order to match the most relevent titles
  inde=[]
  inde = inx[-10:][::-1]

  #saving the movie title in results using the index
  results = movie.title.iloc[inde]
  print(results)





In [ ]:
get_recommendation('Anabelle')


23324                            Elvis and Anabelle (2007)
62422                       Women of Devil's Island (1962)
20815                                   High School (2010)
20813    2013 Rock and Roll Hall of Fame Induction Cere...
20812                                White Reindeer (2013)
20811                             Heiter bis wolkig (2012)
20810      Fly Away (Bis zum Horizont, dann links!) (2012)
20809         Open Up to Me (Kerron sinulle kaiken) (2013)
20808                                Apnea (Apnoia) (2010)
20807              Fuck You, Goethe (Fack Ju Göhte) (2013)
Name: title, dtype: object


In [ ]:
get_recommendation('thor')

40293                                     I Am Thor (2015)
16312                                          Thor (2011)
25069                                Thor: Ragnarok (2017)
20513                          Thor: The Dark World (2013)
43265    While You Were Fighting: A Thor Mockumentary (...
20841          Highlander: The Search for Vengeance (2007)
20801                               As Cool As I Am (2013)
20812                                White Reindeer (2013)
20811                             Heiter bis wolkig (2012)
20810      Fly Away (Bis zum Horizont, dann links!) (2012)
Name: title, dtype: object


In [ ]:
get_recommendation('ponniyin selvan')

62422                       Women of Devil's Island (1962)
20786                                    Green Hell (1940)
20814                                        Gorko! (2013)
20813    2013 Rock and Roll Hall of Fame Induction Cere...
20812                                White Reindeer (2013)
20811                             Heiter bis wolkig (2012)
20810      Fly Away (Bis zum Horizont, dann links!) (2012)
20809         Open Up to Me (Kerron sinulle kaiken) (2013)
20808                                Apnea (Apnoia) (2010)
20807              Fuck You, Goethe (Fack Ju Göhte) (2013)
Name: title, dtype: object




> **This is content based recommendation model and the filtering is based on the title.** 

> **Pros of this model are:**


1.   It'll provide output even when we provide the title in lower case letters
2.   It'll try to provide the recommendations even if the title is not available in the dataset.

> **Cons**


1.   Recommendation is purely based on movie title only. So if an user is trying to watch similar movie based on genre/overview/storyline... this is not the good one






---





`Note: If there is any other column which provides the description of the movie in the dataset, we can use that so that the recommendations will be even much better`




